In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
kozig_uri = u'https://kozigallas.gov.hu/pages/jobviewer.aspx?ID={}'
chrome_header = ('Mozilla/5.0 (Windows NT 6.2; Win64; x64) '
                 'AppleWebKit/537.36 (KHTML, like Gecko) '
                 'Chrome/32.0.1667.0 Safari/537.36')

In [ ]:
def init_session():
    session = requests.Session()
    response = session.get('https://kozigallas.gov.hu/publicsearch.aspx',
                           headers={'User-agent': chrome_header})
    return session

In [ ]:
def get_page(session, url):
    return session.get(url, headers={'User-agent': chrome_header})

In [ ]:
def get_joblists(session):
    response = get_page(session, 'https://kozigallas.gov.hu/publicsearch.aspx')
    soup = BeautifulSoup(response.content, 'html.parser')
    
    max_page = soup.find('span', {'id': ('ctl00_ContentPlaceHolder1_'
                                         'JobSearchForm1_JobList1_lblPageCount2')})
    max_page = int(max_page.getText().split(':')[-1].strip())
    print('Found {} page.'.format(max_page))
    
    search_uri = u'https://kozigallas.gov.hu/publicsearch.aspx?p={}'
    return [search_uri.format(page) for page in range(1, max_page+1)]

In [ ]:
def extract_jobs(response):
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('div', {'class': 'joblist'}).find('table')
    df = pd.read_html(table.prettify(), header=0)[0]
    df = df[df.columns[:-2]]
    df['job_id'] = [jid.get('href').split("'")[1] 
                    for jid in soup.findAll('a', {'class': 'jobapplication'})]
    return df

In [ ]:
def read_job(session, job_id):
    print '.',
    response = session.get(kozig_uri.format(job_id))
    soup = BeautifulSoup(response.content, 'html.parser')
    tag = soup.find('p', {'class': 'MsoNormal'})
    return ' '.join([tag.getText().strip() for tag in soup.findAll('p', {'class': 'MsoNormal'})])

In [ ]:
def scrape():
    with init_session() as session:
        print('joblist download')
        dfs = [extract_jobs(get_page(session, joblist))
               for joblist in get_joblists(session)]
        print('df concat')
        dfs = pd.concat(dfs)
        print('downloading {} jobs'.format(len(dfs)))
        dfs['text'] = dfs.apply(lambda row: read_job(session, row['job_id']),
                                axis=1)
        print('done.')
    return dfs

In [ ]:
df = scrape()

In [ ]:
df.to_csv('data/kozigallas20170727.csv', encoding='utf-8')
df.to_excel('data/kozigallas20170727.xlsx', encoding='utf-8')

---

# Preprocessing data

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('/Users/fulibacsi/workspace/DATA/kozigallas20170725.xlsx').set_index('job_id')

In [ ]:
def clear(text):
    unwanted = ['\xa0', '\r', '\n']
    cleared = ''.join([char for char in text if char not in unwanted])
    return ' '.join(cleared.split())


def process(itemlist):
    items = itemlist.strip().split('•')
    return [item.strip() for item in items if item]


splitters = {
    'feladat': 'A munkakörbe tartozó, illetve a vezetői megbízással járó lényeges feladatok:',
    'illetmeny': 'Illetmény és juttatások:', 
    'feltetel': 'Pályázati feltételek:',
    'elony':    'A pályázat elbírálásánál előnyt jelent:',
    'igazolas': 'A pályázat részeként benyújtandó iratok, igazolások:', 
    'kompetencia': 'Előnyt jelentő kompetenciák:',
    'idopont': 'A munkakör betölthetőségének időpontja:'
}


def generate_order(text):
    order = [(key, text.find(string)) for key, string in splitters.items() if not text.find(string) == -1]
    order = sorted(order, key=lambda x: x[1])
    return [key for key, index in order]
    
    
def extract(text):
    text = clear(text)
    order = generate_order(text)
    data = {key: '' for key, string in splitters.items() 
            if key not in order}

    for start, end in zip(order, order[1:]):
        selected = text.split(splitters[start])
        item = selected[-1].split(splitters[end])
            
        data[start] = process(item[0])
    
    return data

In [ ]:
data['extracted'] = data.text.apply(extract)

In [ ]:
tf = pd.DataFrame([
        {'id': 1, 'nested': {'a': 1, 'b': 2} },
        {'id': 2, 'nested': {'a': 2, 'b': 4} },
        {'id': 3, 'nested': {'a': 3, 'b': 6} },
        {'id': 4, 'nested': {'a': 4}},
    ])

def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

unpack(tf, 'nested', 0)